In [ ]:
# You can train neural networks to convert a given dataset of "what you know" to a dataset of "what you want to know"
# Basically, you can train the network to interpret observations.
# First, convert the observation dataset into matrices so the information is interpretable for the network.
    # Convention: use one row for one observation (each set of on/off lights on a 3-light streetlight) 
    # and one column per observed item (whether each light in the set is on or off). 
    # Ideally, you want a "lossless representation" - the data and the matrix can be perfectly converted between each other.
    
import numpy as np

# input data pattern
# 0 = light is off, 1 = light is on in a 3-light horizontal stoplight at a crosswalk
streetlights = np.array([[1, 0, 1],
                       [0, 1, 1],
                       [0, 0, 1],
                       [1, 1, 1],
                       [0, 1, 1],
                       [1, 0, 1]])
# output data pattern 
# 0 = stop, 1 = walk
walk_vs_stop = np.array([[0],
                        [1],
                        [0],
                        [1],
                        [1],
                        [0]])

# First, we can turn streetlights into walk_vs_stop with a neural network, as before.
